In [2]:
import os
os.chdir('..')
%load_ext autoreload
%autoreload 2

In [164]:
import numpy as np, pandas as pd
import savReaderWriter as spss
from pathlib import Path
pd.set_option('display.max_columns', 100)

In [222]:
def get_columns(file):
    with spss.SavHeaderReader(file) as header:
        labels = header.varLabels
    labels = [e.decode('latin-1') for e in labels.values()]
    return labels

In [6]:
RAW = Path('data/raw/encuesta_relaciones_sociales/')
!ls {RAW}

503-Modulo757  503-Modulo761  503-Modulo765  503-Modulo769
503-Modulo758  503-Modulo762  503-Modulo766  503-Modulo770
503-Modulo759  503-Modulo763  503-Modulo767  503-Modulo771
503-Modulo760  503-Modulo764  503-Modulo768  url.txt


In [235]:
files = [(e.name, [f for f in e.iterdir() if f.suffix == '.sav'][0]) for e in RAW.iterdir() if e.is_dir()]

In [267]:
lines = sum([['-'*80, i, '-'*80]+get_columns(j) for i,j in files], [])
Path('/home/kopita/Desktop/columns.txt').open('w').write('\n'.join(lines))

470446

# Read files

In [57]:
f = '/home/kopita/github/peru-stats/data/raw/encuesta_relaciones_sociales/503-Modulo757/01_CRS01_CAP100.sav'

In [149]:
import savReaderWriter as spss

In [165]:
raw_data = spss.SavReader(f, returnHeader=True)

In [168]:
raw_data.valueLabels

{b'RFINAL': {1.0: b'Completa',
  2.0: b'Incompleta',
  3.0: b'Ausente vivienda',
  4.0: b'Ausente informante',
  5.0: b'Rechazo vivienda',
  6.0: b'Rechazo informante',
  7.0: b'Hogar solo varones',
  8.0: b'Persona con discapacidad',
  9.0: b'Vivienda desocupada',
  10.0: b'No se inicio entrevista',
  11.0: b'Otro'},
 b'C1P101': {1.0: b'Casa independiente',
  2.0: b'Departamento en edif\xedcio',
  3.0: b'Vivienda en quinta',
  4.0: b'Vivienda en casa de vecindad (callej\xf3n, solar o corral\xf3n)',
  5.0: b'Choza o caba\xf1a',
  6.0: b'Vivienda improvisada',
  7.0: b'Local no destinado para habitaci\xf3n humana',
  8.0: b'\xbfOtro?'},
 b'C1P102': {1.0: b'Ladrillo o bloque de cemento?',
  2.0: b'Piedra, sillar con cal o cemento?',
  3.0: b'Adobe?',
  4.0: b'Tapia?',
  5.0: b'Quincha (ca\xf1a con barro)?',
  6.0: b'Piedra con barro?',
  7.0: b'Madera?',
  8.0: b'Estera?',
  9.0: b'Otro material?'},
 b'C1P103': {1.0: b'Concreto armado?',
  2.0: b'Madera?',
  3.0: b'Tejas?',
  4.0: b'Plan

In [152]:
raw_data_list = list(raw_data)

In [154]:
data = pd.DataFrame(raw_data_list)

In [156]:
data = data.rename(columns=data.loc[0]).iloc[1:]

In [157]:
data

,b'CONGLOMERADO',b'NSELV',b'CCDD',b'CCPP',b'CCDI',b'CODCCPP',b'TOHOGAR',b'HOGARN',b'RFINAL',b'C1P101',b'C1P102',b'C1P103',b'C1P104',b'C1P105',b'C1P106',b'C1P107',b'C1P108',b'C1P109',b'C1P110_1',b'C1P110_2',b'C1P110_3',b'C1P110_4',b'C1P110_5',b'C1P110_6',b'C1P110_7',b'C1P111',b'C1P112_1_COD',b'C1P112_2_COD',b'C1P112_3_COD',b'v1',b'FACTOR_VIVIENDA'
1,b'00471',b'0005',b'20',b'07',b'01',b'0001',1,1,1,1,1,4,6,1,1,1,2,1,2,2,1,2,2,2,1,2,b'',b'',b'',1,4749
2,b'00471',b'0012',b'20',b'07',b'01',b'0001',1,1,1,1,9,4,6,1,7,8,3,2,2,2,2,2,1,2,1,2,b'',b'',b'',1,4749
3,b'00471',b'0019',b'20',b'07',b'01',b'0001',1,1,1,1,1,4,6,1,1,1,3,2,2,2,2,2,2,2,1,2,b'',b'',b'',1,4749
4,b'00471',b'0034',b'20',b'07',b'01',b'0001',1,1,1,1,1,4,5,1,1,1,3,2,1,2,2,2,1,2,1,2,b'',b'',b'',1,4749
5,b'00471',b'0041',b'20',b'07',b'01',b'0001',1,1,1,1,1,4,5,1,1,1,4,3,1,2,2,2,1,2,1,2,b'',b'',b'',1,4749
6,b'00471',b'0053',b'20',b'07',b'01',b'0001',1,1,1,1,1,4,5,1,1,1,3,2,2,2,2,2,2,2,1,2,b'',b'',b'',1,4749
7,b'00471',b'0075',b'20',b'07',b'01',b'0001',1,1,1,1,9,4,3,1,4,8,3,1,2,2,2,2,1,2,1,2,b'',b'',b'',1,4749
8,b'00471',b'0077',b'20',b'07',b'01',b'0001',1,1,1,1,1,4,5,2,1,1,3,1,1,1,2,2,1,2,1,2,b'',b'',b'',1,4749
9,b'00687',b'0009',b'20',b'05',b'07',b'0002',1,1,1,1,1,4,3,1,1,1,3,2,1,2,2,2,1,2,1,2,b'',b'',b'',1,4749
10,b'00687',b'0021',b'20',b'05',b'07',b'0002',1,1,1,1,1,4,5,1,1,1,4,3,1,2,1,1,1,2,1,2,b'',b'',b'',1,4749


In [ ]:
with spss.SavReader

In [60]:
with SavReader(f) as reader:
    header = reader.header
#     for line in reader:
#         process(line)

In [62]:
with SavReader(f) as reader:
    records = reader.all()

In [138]:
with SavHeaderReader(f) as header:
    metadata = header.all()
    report = str(header)


In [146]:
with SavReaderNp(f) as reader:
    records = reader.all()
# df = pd.DataFrame(records)
# df.head()

ValueError: title already used as a name or title.

In [147]:
%debug

> /home/kopita/anaconda3/envs/ml/lib/python3.6/site-packages/savReaderWriter/savReaderNp.py(376)trunc_dtype()
    374                    i, v in enumerate(self.uvarNames)]
    375         obj = dict(names=self.uvarNames, formats=formats, titles=self._titles)
--> 376         return np.dtype(obj)
    377 
    378     @memoized_property

ipdb> obj
{'names': ['CONGLOMERADO', 'NSELV', 'CCDD', 'CCPP', 'CCDI', 'CODCCPP', 'TOHOGAR', 'HOGARN', 'RFINAL', 'C1P101', 'C1P102', 'C1P103', 'C1P104', 'C1P105', 'C1P106', 'C1P107', 'C1P108', 'C1P109', 'C1P110_1', 'C1P110_2', 'C1P110_3', 'C1P110_4', 'C1P110_5', 'C1P110_6', 'C1P110_7', 'C1P111', 'C1P112_1_COD', 'C1P112_2_COD', 'C1P112_3_COD', 'v1', 'FACTOR_VIVIENDA'], 'formats': ['a5', 'a4', 'a2', 'a2', 'a2', 'a4', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'f8', 'a2', 'a2', 'a2', 'f8', 'f8'], 'titles': ['CONGLOMERADO', 'NRO DE SELECCIÓN DE LA VIVIENDA', 'CODIGO DE DEPARTAMENTO', 'CODIG

In [130]:
metadata.varLabels[b'C1P102']

b'EN LA VIVIENDA, \xbfEL MATERIAL DE CONSTRUCCI\xd3N PREDOMINANTE EN LAS PAREDES EXTERIORES ES DE:'

In [160]:
storage = Storage()
data = storage.read(f)
fields = storage.describe(f)

TypeError: cannot use a string pattern on a bytes-like object

In [56]:
fields

{'fields': [{'name': 'CONGLOMERADO',
   'type': 'string',
   'title': 'CONGLOMERADO',
   'spss:format': 'A15'},
  {'name': 'NSELV',
   'type': 'string',
   'title': 'NRO DE SELECCIÓN DE LA VIVIENDA',
   'spss:format': 'A12'},
  {'name': 'CCDD',
   'type': 'string',
   'title': 'CODIGO DE DEPARTAMENTO',
   'spss:format': 'A6'},
  {'name': 'CCPP',
   'type': 'string',
   'title': 'CODIGO DE PROVINCIA',
   'spss:format': 'A6'},
  {'name': 'CCDI',
   'type': 'string',
   'title': 'CODIGO DE DISTRITO',
   'spss:format': 'A6'},
  {'name': 'CODCCPP',
   'type': 'string',
   'title': 'CODIGO DE CENTRO POBLADO',
   'spss:format': 'A12'},
  {'name': 'TOHOGAR',
   'type': 'integer',
   'title': 'TOTAL DE HOGARES',
   'spss:format': 'F11'},
  {'name': 'HOGARN',
   'type': 'integer',
   'title': 'HOGAR NRO',
   'spss:format': 'F11'},
  {'name': 'RFINAL',
   'type': 'integer',
   'title': 'RESULTADO FINAL DE LA ENCUESTA',
   'spss:format': 'F11'},
  {'name': 'C1P101',
   'type': 'integer',
   'title